In [1]:
file_path=(r"C:\Users\WIIS\Downloads\SYSTEM SOFTWARE-Ktunotes.in.pdf")

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Extract text from documents
raw_texts = [doc.page_content for doc in pages]

# Split text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents(raw_texts)

In [ ]:
#initializing the embedding model
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
#Embedding and creating the vector store
from langchain_chroma import Chroma

db = Chroma.from_documents(texts,embeddings_model)

In [30]:
retriever = db.as_retriever(search_kwargs={"filter":{"id":"1"}})

In [ ]:
# Initialize the Hugging Face model
from langchain_huggingface import ChatHuggingFace
from langchain_community.chat_models import ChatHuggingFace as CommunityChatHuggingFace
from huggingface_hub import InferenceClient
from langchain_huggingface.llms import HuggingFaceEndpoint

HF_TOKEN="Your HF Token"


# Create endpoint LLM correctly
llm_endpoint = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token=HF_TOKEN,
    task="text-generation",
    temperature=0.3,         
    max_new_tokens=1024       

# Wrap into chat model
llm = ChatHuggingFace(llm=llm_endpoint)


In [37]:
# Define the prompt template
template = """
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create the prompt and output parser
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# Define the chain (pipeline)
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [ ]:
# Test the chain with a query
print(chain.invoke("list the course outcomes"))